# 🚀 Tecton Quickstart
---

Tecton helps you build and productionize AI applications by making it easy to define, test, and deploy features for model training and serving.

Let's take a quick look at how you can build a low-latency streaming feature for a fraud detection use case using nothing but Python.

For an end-to-end tutorial that takes you from raw data to a real-time model, check out Building a Production AI Application with Tecton (tecton-02-building.ipynb.)

## ⚙️ Install Tecton

In [1]:
%pip install 'tecton[rift]' --quiet --upgrade

Note: you may need to restart the kernel to use updated packages.


## ✅ Log in to Tecton

Make sure to hit `enter` after pasting in your authentication token.

In [2]:
import tecton

tecton.login("demo-pangolin.tecton.ai")
tecton.set_validation_mode("auto")

tecton.version.summary()

Already logged in to https://demo-pangolin.tecton.ai as UserProfile(name='Jonathan Varley', email='jon@tecton.ai', id='00ut35dahebreB27E357'). To switch users, run `tecton.logout` then `tecton.login`
Version: 0.9.12
Git Commit: 7b1322f6df430b497a8fd0535186da3bf3ee6612
Build Datetime: 2024-06-25T14:37:08


## 🧪 Define and test a streaming feature

Using Tecton's Feature Engineering Framework we will define 3 new features for our fraud detection model:

- A user's total transaction amount in the last 1 minute
- A user's total transaction amount in the last 1 hour
- A user's total transaction amount in the last 30 days

To do so, we will first define a [Stream Source](https://docs.tecton.ai/docs/defining-features/data-sources/creating-a-data-source/creating-and-testing-a-push-source) which tells Tecton where to retrieve events online and offline. For the online path, we've configured Tecton to accept real-time events via an HTTP ingestion API which we will try sending records to in the next step. For the offline path, we've pointed Tecton to an S3 path that contains a historical record of our stream. Tecton uses this path for offline development and backfills.

Next we define a [Stream Feature View](https://docs.tecton.ai/docs/defining-features/feature-views/stream-feature-view/stream-feature-view-with-rift) which can create one or more features via transformations against the Stream Source. In a Stream Feature View, we can run Python transformations on incoming records, and optionally apply time-windowed [aggregations](https://docs.tecton.ai/docs/defining-features/feature-views/aggregation-engine) via the `aggregations` parameter. These transformations run identically online and offline which is critical for preventing skew.

Lastly, we can immediately test our features directly in our notebook via the `get_features_in_range` method. Try it out!

In [3]:
from tecton import *
from tecton.types import *
from datetime import datetime, timedelta


transactions_stream = StreamSource(
    name="transactions_stream",
    stream_config=PushConfig(),
    batch_config=FileConfig(
        uri="s3://mft-porter-data/tutorials/transactions.pq",
        file_format="parquet",
        timestamp_field="timestamp",
    ),
    schema=[Field("user_id", String), Field("timestamp", Timestamp), Field("amount", Float64)],
)


user = Entity(name="user", join_keys=["user_id"])


@stream_feature_view(
    source=transactions_stream,
    entities=[user],
    mode="pandas",
    aggregations=[
        Aggregation(function="sum", column="amount", time_window=timedelta(minutes=1)),
        Aggregation(function="sum", column="amount", time_window=timedelta(hours=1)),
        Aggregation(function="sum", column="amount", time_window=timedelta(days=30)),
    ],
    schema=[Field("user_id", String), Field("timestamp", Timestamp), Field("amount", Float64)],
)
def user_transaction_amount_totals(transactions_stream):
    return transactions_stream[["user_id", "timestamp", "amount"]]


# Test the feature locally using historical data
df = (
    user_transaction_amount_totals.get_features_in_range(start_time=datetime(2022, 1, 1), end_time=datetime(2022, 2, 1))
    .to_pandas()
    .fillna(0)
)
df.head(5)

StreamFeatureView 'user_transaction_amount_totals': Validating 3 dependencies.
    StreamSource 'transactions_stream': Deriving schema.
    StreamSource 'transactions_stream': Successfully validated.
    Entity 'user': Successfully validated.
    Transformation 'user_transaction_amount_totals': Successfully validated.
StreamFeatureView 'user_transaction_amount_totals': Successfully validated.


,user_id,amount_sum_1m_continuous,amount_sum_1h_continuous,amount_sum_30d_continuous,_valid_to,_valid_from
0,user_4063572189,0.0,868.62,4008.09,2022-01-04,2022-01-03
1,user_9341009982,0.0,41.17,6252.08,2022-01-08,2022-01-07
2,user_1997016327,0.0,0.00,7106.46,2022-01-13,2022-01-12
3,user_2210887384,0.0,0.00,2940.89,2022-01-24,2022-01-23
4,user_2392573024,0.0,0.00,11524.19,2022-01-21,2022-01-20


## ⚡️ Ingest data and retrieve updated feature values in real-time

Tecton objects get registered via a [declarative workflow](https://docs.tecton.ai/docs/the-feature-development-workflow). Features are defined as code in a repo and applied to a workspace (like a project) in a Tecton account using the Tecton CLI. This declarative workflow enables productionization best practices such as "features as code," CI/CD, and unit testing.

We've gone ahead and [registered these features](https://demo-pangolin.tecton.ai/app/repo/prod/features/user_transaction_amount_totals/pipeline) so you can try ingesting new events and querying for online features. After features get registered with a workspace, Tecton handles the [backfilling and ongoing materialization](https://demo-pangolin.tecton.ai/app/repo/prod/features/user_transaction_amount_totals/materialization) of data to the offline and online store for training and serving.

**This step requires generating an API key, which you can do [here](https://demo-pangolin.tecton.ai/app/settings/accounts-and-access/service-accounts?create-service-account=true). Copy the generated key and paste it in below.**

In [5]:
import random, string

tecton.set_credentials(tecton_api_key="your-api-key")  # replace with your API key

# Fetch registered Tecton objects
ws = tecton.get_workspace("prod")
ds = ws.get_data_source("transactions_stream")
fv = ws.get_feature_view("user_transaction_amount_totals")

# Generate a random user_id for the next step
user_id = "user_" + "".join(random.choices(string.digits, k=7))
print("Generated random user id: " + user_id)

Generated random user id: user_2269114


### 🔥 Run this repeatedly and watch the features update!

**Note: It may take a few minutes for your API key permissions to update.**

In [7]:

# Ingest a new transaction with any amount you want
record = ds.ingest({"user_id": user_id, "timestamp": datetime.utcnow(), "amount": 100})
print(f"Ingested record for '{user_id}':")
print(record)

# Read updated features via Tecton's HTTP API
features = fv.get_online_features(join_keys={"user_id": user_id}).to_dict()
print(f"\nUpdated features for '{user_id}':")
print(features)

Ingested record for 'user_2269114':
{'workspaceName': 'prod', 'ingestMetrics': {'featureViewIngestMetrics': [{'featureViewName': 'user_transaction_amount_totals', 'onlineRecordIngestCount': '1', 'featureViewId': '73ed15b75c519022d5508f86de28c9b9'}]}}

Updated features for 'user_2269114':
{'amount_sum_1h_continuous': 200.0, 'amount_sum_1m_continuous': 200.0, 'amount_sum_30d_continuous': 200.0}


## ⭐️ Conclusion

Now that you've seen how easy it is to build real-time features with Tecton, check out our Building a Production AI Application tutorial to see how you can productionize an end-to-end online AI Application in just 30 minutes.